In [46]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [47]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [48]:
spark.version

'3.5.0'

In [49]:
from pyspark.sql import types

In [50]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
     types.StructField('Affiliated_base_number', types.StringType(), True),
])

In [52]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('data/raw/fhv/2019/10/fhv_tripdata_2019-10.csv')

df = df.repartition(6)

df.write.parquet('data/pq/fhv/2019/10/', mode='overwrite')

In [53]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00536|2019-10-01 14:49:51|2019-10-01 14:58:05|         264|         165|   NULL|                B00536|
|              B03156|2019-10-02 12:38:00|2019-10-02 12:59:00|         238|          43|   NULL|                B03156|
|              B02111|2019-10-01 12:18:01|2019-10-01 12:21:40|           7|         132|   NULL|                B02111|
|              B00789|2019-10-01 13:28:41|2019-10-01 13:50:43|         264|         264|   NULL|                B00789|
|              B02735|2019-10-02 22:21:50|2019-10-02 22:43:19|         264|          69|   NULL|                B03136|
|              B02101|2019-10-01 12:18:3

In [54]:
df = spark.read.parquet('data/pq/fhv/2019/10/')

In [55]:
from pyspark.sql import functions as F

In [56]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter("pickup_date = '2019-10-15'") \
    .count()

62610

In [57]:
df.createOrReplaceTempView('fhv_2019_10')

In [58]:
spark.sql("""
SELECT
    COUNT(1)
FROM 
    fhv_2019_10
WHERE
    to_date(pickup_datetime) = '2019-10-15';
""").show()

+--------+
|count(1)|
+--------+
|   62610|
+--------+



In [59]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [60]:
df \
    .withColumn('duration', (df.dropoff_datetime.cast('long') - df.pickup_datetime.cast('long')) / 3600) \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(10) \
    .show()

+-----------+------------------+
|pickup_date|     max(duration)|
+-----------+------------------+
| 2019-10-28|          631152.5|
| 2019-10-11|          631152.5|
| 2019-10-31| 87672.44083333333|
| 2019-10-01| 70128.02805555555|
| 2019-10-17|            8794.0|
| 2019-10-26| 8785.166666666666|
| 2019-10-30|1464.5344444444445|
| 2019-10-25|1057.8266666666666|
| 2019-10-02| 770.2313888888889|
| 2019-10-23| 746.6166666666667|
+-----------+------------------+



In [61]:
spark.sql("""
SELECT
    to_date(pickup_datetime) AS pickup_date,
    MAX((CAST(dropoff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / 3600) AS duration
FROM 
    fhv_2019_10
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 10;
""").show()

+-----------+------------------+
|pickup_date|          duration|
+-----------+------------------+
| 2019-10-28|          631152.5|
| 2019-10-11|          631152.5|
| 2019-10-31| 87672.44083333333|
| 2019-10-01| 70128.02805555555|
| 2019-10-17|            8794.0|
| 2019-10-26| 8785.166666666666|
| 2019-10-30|1464.5344444444445|
| 2019-10-25|1057.8266666666666|
| 2019-10-02| 770.2313888888889|
| 2019-10-23| 746.6166666666667|
+-----------+------------------+



In [62]:
df_zones = spark.read.parquet('zones')

In [63]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [64]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [65]:
df_zones.createOrReplaceTempView('zones')

In [66]:
spark.sql("""
SELECT
    pul.Zone AS pu_zone,
    COUNT(1)
FROM 
    fhv_2019_10 fhv LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID                  
GROUP BY 
    1
ORDER BY
    2 ASC
LIMIT 5;
""").show()

+--------------------+--------+
|             pu_zone|count(1)|
+--------------------+--------+
|         Jamaica Bay|       1|
|Governor's Island...|       2|
| Green-Wood Cemetery|       5|
|       Broad Channel|       8|
|     Highbridge Park|      14|
+--------------------+--------+

